### Loading Packages

In [83]:
library(dplyr)

current_date <- as.Date("2025-01-06")


In [84]:
cbbdata::cbd_login(username = "Naki", password = "Duke@123")
data <- cbbdata::cbd_torvik_team_factors(year = 2025, no_bias = TRUE) 
print(data)

API Key set!# A tibble: 364 x 31
   team   conf  games  wins losses adj_t adj_o adj_d barthag   efg def_efg   ftr
   <chr>  <chr> <dbl> <dbl>  <dbl> <dbl> <dbl> <dbl>   <dbl> <dbl>   <dbl> <dbl>
 1 Auburn SEC      13    12      1  69.2  134.  94.5   0.983  58.8    43.9  29.4
 2 Houst~ B12      12     9      3  63.1  119.  84.2   0.981  52.5    39.6  27.5
 3 Duke   ACC      13    11      2  67    120.  86.9   0.975  55.5    42    29.4
 4 Tenne~ SEC      13    13      0  66.7  118.  86.5   0.973  56.6    40.6  34  
 5 Flori~ SEC      13    13      0  71    123.  93.3   0.958  54.6    41.7  31  
 6 Gonza~ WCC      14    10      4  71.7  124.  94.7   0.955  55.5    46.7  31.5
 7 Iowa ~ B12      12    11      1  71    126.  96.8   0.953  56.5    48.4  39.6
 8 Maryl~ B10      13    11      2  71.3  117.  90.7   0.950  56.9    45.2  29.7
 9 Illin~ B10      12     9      3  73.1  117.  90.6   0.949  53.4    41.8  40.2
10 Alaba~ SEC      13    11      2  75.1  124.  96.9   0.947  55.1    45.6  

## Extract ACC and B10 Teams from list

In [85]:
# # Filter teams in ACC and B10 and extract into separate lists
acc_teams <- data |>
  filter(conf == "ACC") |>
  pull(team) # Extract the 'team' column as a list

b10_teams <- data |>
  filter(conf == "B10") |>
  pull(team) # Extract the 'team' column as a list

SEC_teams <- data |>
  filter(conf == "SEC") |>
  pull(team) # Extract the 'team' column as a list

# espn_top25_week8 <- c(
#   "Tennessee", "Auburn", "Iowa St.", "Duke", "Alabama", "Florida", 
#   "Kansas", "Marquette", "Oregon", "Kentucky", "Connecticut", "Oklahoma", 
#   "Texas A&M", "Gonzaga", "Houston", "Mississippi", "Cincinnati", 
#   "Michigan St.", "Mississippi St.", "San Diego St.", 
#   "Purdue", "UCLA", "Arkansas", "Illinois", "Baylor"
# )

# extra_games <- c("Colorado St.", "Georgia St.", "North Carolina Central", "Iowa St.", 
#   "Richmond", "Boise St.", "Nevada", "Eastern Kentucky",  "Iona", "Texas St.", 
#   "Army", "Northeastern", "Northwestern"
#   )

unique_teams <- unique(c(acc_teams, b10_teams, SEC_teams))

# print(acc_teams)
# print(b10_teams)
# print(b12_teams)
# print(espn_top25_week8)
# print(unique_teams)

#unique_teams <- c("Duke", "North Carolina", "North Carolina St.")

## Extract Relevant Factors

In [86]:
selected_data_columns <- data |> 
  dplyr::select(Team = team, 
                ADJO = adj_o, 
                ADJD = adj_d,
                EFG_pct = efg, 
                TO_Pct = tov_rate, 
                OR_Pct = oreb_rate,
                FTR_Pct = ftr, 
                ThreePointer_Pct = three_pt_pct)

print(selected_data_columns)
print(which(is.na(selected_data_columns), arr.ind = TRUE))

# A tibble: 364 x 8
   Team       ADJO  ADJD EFG_pct TO_Pct OR_Pct FTR_Pct ThreePointer_Pct
   <chr>     <dbl> <dbl>   <dbl>  <dbl>  <dbl>   <dbl>            <dbl>
 1 Auburn     134.  94.5    58.8   12.8   36.2    29.4             37.1
 2 Houston    119.  84.2    52.5   14.8   36.6    27.5             40.6
 3 Duke       120.  86.9    55.5   15.4   34.5    29.4             35.6
 4 Tennessee  118.  86.5    56.6   16.2   38.4    34               35.5
 5 Florida    123.  93.3    54.6   15.4   41.4    31               34.4
 6 Gonzaga    124.  94.7    55.5   14.3   32.8    31.5             33.3
 7 Iowa St.   126.  96.8    56.5   13.4   35.1    39.6             34.2
 8 Maryland   117.  90.7    56.9   13.7   34.5    29.7             36.6
 9 Illinois   117.  90.6    53.4   15.9   36.6    40.2             33.4
10 Alabama    124.  96.9    55.1   16.8   36.8    41.2             31.6
# i 354 more rows
     row col


## Cleaning Season Schedule

In [87]:
all_games <- cbbdata::cbd_torvik_season_schedule(year = 2025)
all_games <- all_games |>
  mutate(Location = ifelse(neutral, "N", "H")) |>
  select(-neutral, -type, -year, -game_id) |>
  rename(Team = home,
         Opponent = away,
         )

print(which(is.na(all_games), arr.ind = TRUE))
print(all_games)

     row col
            date              Team         Opponent Location
          <Date>            <char>           <char>   <char>
   1: 2024-11-04             Tulsa Northeastern St.        H
   2: 2024-11-04         Tennessee     Gardner Webb        H
   3: 2024-11-04 Houston Christian            Avila        H
   4: 2024-11-04         Stonehill   Framingham St.        H
   5: 2024-11-04    North Carolina             Elon        H
  ---                                                       
5846: 2025-03-09  Florida Atlantic    East Carolina        H
5847: 2025-03-09          Nebraska             Iowa        H
5848: 2025-03-09            Temple      North Texas        H
5849: 2025-03-09       Wichita St.            Tulsa        H
5850: 2025-03-09        Washington           Oregon        H


## Finding Games for Particluar Data

In [88]:
upcoming_games <- all_games |>
  filter(date == current_date)
print(upcoming_games)

required_upcoming_games <- upcoming_games |>
  filter(Team %in% unique_teams | Opponent %in% unique_teams)
print(required_upcoming_games)

          date                   Team               Opponent Location
        <Date>                 <char>                 <char>   <char>
 1: 2025-01-06             Coppin St.     South Carolina St.        H
 2: 2025-01-06 Southeastern Louisiana   UT Rio Grande Valley        H
 3: 2025-01-06            Wichita St.          South Florida        H
 4: 2025-01-06            New Orleans Texas A&M Corpus Chris        H
 5: 2025-01-06              Minnesota               Ohio St.        H
 6: 2025-01-06            Alabama A&M Mississippi Valley St.        H
 7: 2025-01-06             Morgan St. North Carolina Central        H
 8: 2025-01-06                Houston                    TCU        H
 9: 2025-01-06           Delaware St.            Norfolk St.        H
10: 2025-01-06                Rutgers              Wisconsin        H
11: 2025-01-06            Alabama St.    Arkansas Pine Bluff        H
12: 2025-01-06     Texas A&M Commerce           Nicholls St.        H
13: 2025-01-06      

In [89]:
# Extract stats for Team and Opponent
team_data <- required_upcoming_games |>
  left_join(selected_data_columns, by = c("Team" = "Team"))

opponent_data <- required_upcoming_games |>
  left_join(selected_data_columns, by = c("Opponent" = "Team")) |>
  rename_with(~ paste("Opp_", ., sep = ""), everything())

# Rename and remove unwanted columns
opponent_data <- opponent_data |>
  rename(Opponent = Opp_Opponent, Team = Opp_Team) |>
  select(-c("Opp_date", "Opp_Location"))

# Combine both datasets
final_data <- team_data |>
  left_join(opponent_data, by = c("Team", "Opponent"))

# Print the final result
print(final_data)

setdiff(required_upcoming_games$Team, selected_data_columns$Team)
setdiff(required_upcoming_games$Opponent, selected_data_columns$Team)


         date      Team  Opponent Location     ADJO     ADJD EFG_pct TO_Pct
       <Date>    <char>    <char>   <char>    <num>    <num>   <num>  <num>
1: 2025-01-06 Minnesota  Ohio St.        H 103.9662 101.7149    50.1   15.4
2: 2025-01-06   Rutgers Wisconsin        H 113.2308 103.1023    52.1   14.9
   OR_Pct FTR_Pct ThreePointer_Pct Opp_ADJO  Opp_ADJD Opp_EFG_pct Opp_TO_Pct
    <num>   <num>            <num>    <num>     <num>       <num>      <num>
1:   29.0    33.9             30.5 115.0569  97.79511        57.7       14.6
2:   30.2    38.7             33.6 120.4040 100.48809        52.3       13.8
   Opp_OR_Pct Opp_FTR_Pct Opp_ThreePointer_Pct
        <num>       <num>                <num>
1:       27.5        37.8                 39.6
2:       29.4        38.2                 32.5


character(0)

character(0)

In [90]:
file_to_write <- "test_file_Jan06.csv"
write.csv(final_data, file_to_write, row.names = FALSE)